In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import chi2_contingency

df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
contingency = pd.crosstab(df.embark_town, df['class'])
contingency

class,First,Second,Third
embark_town,,,
Cherbourg,85,17,66
Queenstown,2,3,72
Southampton,127,164,353


In [3]:
contingency = contingency.to_numpy()

In [4]:
total_samples = np.sum(contingency)

Next I am checking the number of rows in the initial dataset to make sure that they match:

In [5]:
df.shape[0]

891

They don't actually match. To check what's going on, I will check how many rows have an N/A for one of the two columns:

In [6]:
df[['embark_town', 'class']].isna().max(axis=1).sum()

2

Ok, it looks like 2 rows contained an N/A, so they were excluded from the contingency table. 

## Calculate Degrees of Freedom

In [7]:
degrees_of_freedom = (contingency.shape[0] - 1) * (contingency.shape[1] - 1)
degrees_of_freedom

4

## Calculate Expected Values

In [8]:
col_sum = contingency.sum(axis=0)
row_sum = contingency.sum(axis=1)

Next I am calculating the expected values. 

The expected values are calculated for each row, and are what the values would be if there was no relationship between the two variables. 

In the cell below, I am using a matrix multiplication of the sum of the rows and the sum of the columns

In [8]:
expected = np.matmul(row_sum.reshape(-1, 1), col_sum.reshape(1, -1)) / total_samples
expected

array([[ 40.44094488,  34.77165354,  92.78740157],
       [ 18.53543307,  15.93700787,  42.52755906],
       [155.02362205, 133.29133858, 355.68503937]])

In [13]:
chi2 = np.sum(((expected - contingency) ** 2) / expected)
chi2

123.75190952951289

In [11]:
chi2_scipy, p_scipy, degrees_of_freedom_scipy, expected_scipy = chi2_contingency(contingency)

chi2_scipy, p_scipy, degrees_of_freedom_scipy, expected_scipy

(123.75190952951289,
 8.435267819894384e-26,
 4,
 array([[ 40.44094488,  34.77165354,  92.78740157],
        [ 18.53543307,  15.93700787,  42.52755906],
        [155.02362205, 133.29133858, 355.68503937]]))

### Double Check Calculations

In [12]:
assert np.abs(chi2 - chi2_scipy) < 0.0001
assert np.abs(degrees_of_freedom - degrees_of_freedom_scipy) < 0.0001
assert np.sum(np.abs(expected - expected_scipy)) < 0.0001